In [5]:
#!/usr/bin/env python

from parser import *

inputXML = "sample_rawdata"
outputGold = "sample_gold"
outputXML = "sample_train"

# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# TRAIN:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>

parse_folder(inputXML, outputGold, 0)
allSentences, allSurfaceFeatures = parse_folder(inputXML, outputXML, 1)

print allSentences[0][0]



# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# RANDOM NOTES:
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# import script as though it is a module to run functions from it.
# s = f('summary')
# s.text()
# len(s)
# dir(s)
# type(s)

Parsing files in: sample_rawdata ...
[0] Summary for 0a14bbc9ed2e173752d0f6e9b0455cca.xml (John Calvin)  --->  sample_gold/gold_0a14bbc9ed2e173752d0f6e9b0455cca.txt
[1] Summary for 0a20be5b6b253ab99abeadf6a660abf5.xml (El Lissitzky)  --->  sample_gold/gold_0a20be5b6b253ab99abeadf6a660abf5.txt
[2] Summary for 0a36ec48fbe716534f61351534435650.xml (Sonatas and Interludes)  --->  sample_gold/gold_0a36ec48fbe716534f61351534435650.txt
[3] Summary for 0a3bc28026f6c569153e8eea3aa28a5d.xml (The Lord of the Rings (1978 film))  --->  sample_gold/gold_0a3bc28026f6c569153e8eea3aa28a5d.txt
[4] Summary for 0a4f82d285bd0a3136d6e7254f428647.xml (Order of St. Patrick)  --->  sample_gold/gold_0a4f82d285bd0a3136d6e7254f428647.txt
[5] Summary for 0a6fd7fbcf2c2e42a8de54d1f30ac8ab.xml (Lundomys)  --->  sample_gold/gold_0a6fd7fbcf2c2e42a8de54d1f30ac8ab.txt
Parsing files in: sample_rawdata ...
[0] Parsing: 0a14bbc9ed2e173752d0f6e9b0455cca.xml
[1] Parsing: 0a20be5b6b253ab99abeadf6a660abf5.xml
[2] Parsing: 0a36e

In [6]:
allSentences

[{0: 'Calvin was born as Jehan Cauvin on 10 July 1509, in the town of Noyon in the Picardy region of France.',
  1: 'He was the first of four sons who survived infancy.',
  2: 'His father, Grard Cauvin, had a prosperous career as the cathedral notary and registrar to the ecclesiastical court.',
  3: 'He died in his later years, after suffering two years with testicular cancer.',
  4: 'His mother, Jeanne le Franc, was the daughter of an innkeeper from Cambrai.',
  5: "She died a few years after Calvin's birth from breast disease (not breast cancer).",
  6: 'Grard intended his three sonsCharles, Jean, and Antoinefor the priesthood.',
  7: 'Jean was particularly precocious; by age 12, he was employed by the bishop as a clerk and received the tonsure, cutting his hair to symbolise his dedication to the Church.',
  8: 'He also won the patronage of an influential family, the Montmors.',
  9: 'Through their assistance, Calvin was able to attend the Collge de la Marche, in Paris, where he lear